In [1]:
import pandas as pd
import re
import numpy as np
import pdfplumber

In [3]:
def extrair_dados_por_residencia(nome_arquivo):
    with pdfplumber.open(nome_arquivo) as pdf:
        todas_notas = []
        residencia_atual = None
        notas_residencia = []

        for  pagina in pdf.pages:
            
            text = pagina.extract_text()
            linhas = text.strip().split('\n')

            for linha in linhas:
                if linha.startswith("PRM"):
                    if residencia_atual is not None:
                        todas_notas.append((residencia_atual, notas_residencia))
                        notas_residencia = []

                    residencia_atual = linha  # Atualiza o nome da residência
                else:
                    # Adiciona a linha como nota, ignorando a linha de cabeçalho
                    notas_residencia.append(linha)
    
        # Adiciona as últimas notas da última residência
        if residencia_atual is not None and notas_residencia:
            todas_notas.append((residencia_atual, notas_residencia))

    return todas_notas

nome_arquivo_pdf = 'C://Users//joao.salmen//Downloads//ce4ead53-ba9d-4dea-93fb-35432f84a719.pdf'
dados_notas = extrair_dados_por_residencia(nome_arquivo_pdf)


In [4]:
def corrigir_lista_notas(lista):
    lista_corrigida = []
    i = 0

    while i < len(lista):
        # Verificar se a string atual parece ser um nome sem as notas
        if not re.search(r'\d{10}\s(\d+\.\d+\s){5}\d+\.\d+', lista[i]):
            # Verificar se a próxima string na lista contém o padrão de notas e número de inscrição
            if i + 1 < len(lista) and re.search(r'\d{10}\s(\d+\.\d+\s){5}\d+\.\d+', lista[i + 1]):
                # Combinar a string atual com a próxima
                lista_corrigida.append(lista[i] + ' ' + lista[i + 1])
                i += 2  # Pular a próxima string, pois já foi combinada
            else:
                # Se a string atual não se encaixa em nenhum padrão conhecido, mantê-la como está
                lista_corrigida.append(lista[i])
                i += 1
        else:
            # Se a string atual já está no formato correto, mantê-la como está
            lista_corrigida.append(lista[i])
            i += 1

    return lista_corrigida

In [5]:
def filtrar_linhas_com_regex(lista):
    # Regex para filtrar linhas
    regex = r'.+\d{10}\s(\d+\.\d+\s){5}\d+\.\d+'

    # Filtrar apenas as linhas que correspondem ao padrão
    linhas_filtradas = [linha for linha in lista if re.match(regex, linha)]
    return linhas_filtradas

In [6]:
import re

def separar_por_maiusculas(nome):
    # Separando o nome pelas letras maiúsculas
    return re.sub(r"([A-Z])", r" \1", nome).strip()

def criar_dicionario_notas_revisado(lista):
    lista_dicionarios = []

    for item in lista:
        # Encontrando o número de inscrição e as notas com expressões regulares
        match = re.match(r'(.+?) (\d{10}) ((?:\d+\.\d+ ){5}\d+\.\d+)', item)
        
        if match:
            # Extraindo o nome, número de inscrição e notas
            nome_completo, inscricao, notas = match.groups()
            notas = notas.split()

            # Extrair apenas o segundo nome, se disponível
            nome_dividido = nome_completo.split()
            nome = nome_dividido[1] if len(nome_dividido) > 1 else nome_completo

            # Separar o nome pelas letras maiúsculas
            nome = separar_por_maiusculas(nome)

            # Construindo o dicionário
            dicionario = {
                "Nome": nome,
                "Inscrição": inscricao,
                "Clínica Médica": notas[0],
                "Cirurgia Geral": notas[1],
                "Pediatria": notas[2],
                "Ginecologia e Obstetrícia": notas[3],
                "Medicina Preventiva e Social": notas[4],
                "Nota Objetiva": notas[5]
            }
            lista_dicionarios.append(dicionario)

    return lista_dicionarios



In [7]:

residencias_escrita=[]
for residencia in dados_notas:
    fields = {}
    fields['residencia']=residencia[0].replace('–','-').split('-')[-1].strip().replace('/','')
    lista=corrigir_lista_notas(residencia[1])
    lista = filtrar_linhas_com_regex(lista)
    lista=criar_dicionario_notas_revisado(lista)
    fields['dados']=lista
    residencias_escrita.append(fields)
residencias_escrita=residencias_escrita[:29]

In [8]:
arquivo_curriculo = 'C://Users//joao.salmen//Downloads//9efbafa9-5aa4-48c9-99b8-26b948840968.pdf'
arquivo_curriculo = extrair_dados_por_residencia(arquivo_curriculo)

In [9]:
def filtrar_valores_especificos_com_nomes_juntos_v2(lista):
    # Regex adaptada para o novo padrão
    regex = r'.+\d{10}\s\d+\.\d+\s\w+\s\w+'

    # Aplicar a regex diretamente sem juntar os nomes
    valores_filtrados = [item for item in lista if re.match(regex, item)]

    return valores_filtrados

In [10]:
def criar_dicionarios_de_candidatos_v2(lista):
    lista_dicionarios = []

    for item in lista:
        partes = item.split()
        nome = " ".join(partes[:-4])
        inscricao = partes[-4]
        nota_analise_curricular = partes[-3]
        provab = partes[-2]
        prmgfc = partes[-1]

        dicionario_candidato = {
            "Nome": nome,
            "Inscrição": inscricao,
            "Nota da Análise Curricular": nota_analise_curricular,
            "PROVAB": provab,
            "PRMGFC": prmgfc
        }

        lista_dicionarios.append(dicionario_candidato)

    return lista_dicionarios

In [11]:
def filtrar_valores_especificos_com_nomes_juntos_v2(lista):
    # Regex adaptada para o novo padrão
    regex = r'.+\d{10}\s\d+\.\d+\s\w+\s\w+'

    # Aplicar a regex diretamente sem juntar os nomes
    valores_filtrados = [item for item in lista if re.match(regex, item)]

    return valores_filtrados

def criar_dicionarios_de_candidatos_v2(lista):
    lista_dicionarios = []

    for item in lista:
        partes = item.split()
        nome = " ".join(partes[:-4])
        inscricao = partes[-4]
        nota_analise_curricular = partes[-3]
        provab = partes[-2]
        prmgfc = partes[-1]

        dicionario_candidato = {
            "Nome": nome,
            "Inscrição": inscricao,
            "Nota da Análise Curricular": nota_analise_curricular,
            "PROVAB": provab,
            "PRMGFC": prmgfc
        }

        lista_dicionarios.append(dicionario_candidato)

    return lista_dicionarios



In [12]:
# Aplicar o código aos dados
residencias_curriculo = []
for residencia in arquivo_curriculo:
    fields = {}
    fields['residencia'] = residencia[0].replace('–','-').split('-')[-1].strip().replace(' ','').replace('/','')
    lista = filtrar_valores_especificos_com_nomes_juntos_v2(residencia[1])
    lista = criar_dicionarios_de_candidatos_v2(lista)
    fields['dados'] = lista
    residencias_curriculo.append(fields)
residencias_curriculo = residencias_curriculo[:29]

In [43]:
def combinar_dados_e_criar_csvs(lista1, lista2):
    # Dicionário para armazenar os dados combinados
    dados_combinados = {}

    # Processar a primeira lista
    for item in lista1:
        residencia = item['residencia']
        for candidato in item['dados']:
            chave = (residencia, candidato['Inscrição'])
            dados_combinados[chave] = candidato

    # Processar a segunda lista e combinar os dados
    for item in lista2:
        residencia = item['residencia']
        for candidato in item['dados']:
            chave = (residencia, candidato['Inscrição'])
            if chave in dados_combinados:
                dados_combinados[chave].update(candidato)
            else:
                dados_combinados[chave] = candidato

    # Agrupar os dados por residência
    dados_por_residencia = {}
    for chave, valor in dados_combinados.items():
        residencia = chave[0]
        if residencia not in dados_por_residencia:
            dados_por_residencia[residencia] = []
        dados_por_residencia[residencia].append(valor)
    dfs=[]
    # Criar um DataFrame e salvar como CSV para cada residência
    for residencia, dados in dados_por_residencia.items():
        df = pd.DataFrame(dados)
        df['Nota Objetiva'] = pd.to_numeric(df['Nota Objetiva'], errors='coerce')
        df['Nota da Análise Curricular'] = pd.to_numeric(df['Nota da Análise Curricular'], errors='coerce')
        df['nota_final_sem_bonus'] = df['Nota Objetiva']*0.9 + df['Nota da Análise Curricular']*0.1
        df['nota_final_com_bonus'] = np.where((df['PROVAB'].str.contains('Sim')) | (df['PRMGFC'].str.contains('Sim')),  df['nota_final_sem_bonus']*1.1, df['nota_final_sem_bonus'])
        # Remover espaços e caracteres especiais do nome da residência para o nome do arquivo
        nome_arquivo = residencia + '.csv'
        print(residencia)
        dfs.append(df)
        df.to_excel(residencia + '.xlsx', index=False)
    return dfs

In [44]:
dfs = combinar_dados_e_criar_csvs(residencias_curriculo,residencias_escrita)

ACUPUNTURA
ANATOMIAPATOLÓGICA
ANESTESIOLOGIA
CIRURGIACARDIOVASCULAR
CIRURGIAGERAL
CLÍNICAMÉDICA
DERMATOLOGIA
GENÉTICAMÉDICA
GINECOLOGIAEOBSTETRÍCIA
HOMEOPATIA
INFECTOLOGIA
MEDICINADEEMERGÊNCIA
MEDICINADEFAMÍLIAECOMUNIDADE
MEDICINADOTRABALHO
MEDICINADOTRÁFEGO
MEDICINAINTENSIVA
MEDICINANUCLEAR
MEDICINAPREVENTIVAESOCIAL
NEUROCIRURGIA
NEUROLOGIA
OFTALMOLOGIA
ORTOPEDIAETRAUMATOLOGIA
OTORRINOLARINGOLOGIA
PATOLOGIACLÍNICAMEDICINALABORATORIAL
PATOLOGIA
PEDIATRIA
PSIQUIATRIA
RADIOLOGIAEDIAGNÓSTICOPORIMAGEM
RADIOTERAPIA
